## Выполнили: Жерноклеев Дмитрий Дамирович, Индык Илья Андреевич, Реентович Александр Анатольевич

In [2]:
# Токенизированные тексты в нижнем регистре
!wget  https://www.dropbox.com/s/gwfgmomdbetvdye/hpac_lower_tokenized.zip

 .......... .......... 66% 74.0M 23s
646750K .......... .......... .......... .......... .......... 66% 47.7M 23s
646800K .......... .......... .......... .......... .......... 66% 86.7M 23s
646850K .......... .......... .......... .......... .......... 66%  607K 23s
646900K .......... .......... .......... .......... .......... 66% 79.8M 23s
646950K .......... .......... .......... .......... .......... 66%  180M 23s
647000K .......... .......... .......... .......... .......... 66% 60.2M 23s
647050K .......... .......... .......... .......... .......... 66% 51.0M 23s
647100K .......... .......... .......... .......... .......... 66%  177M 23s
647150K .......... .......... .......... .......... .......... 66% 67.6M 23s
647200K .......... .......... .......... .......... .......... 66% 60.4M 23s
647250K .......... .......... .......... .......... .......... 66% 53.4M 23s
647300K .......... .......... .......... .......... .......... 66% 93.6M 23s
647350K .......... .......... .........

In [8]:
# train-test-dev
!wget https://www.dropbox.com/s/3vdz0mouvex8abd/hpac_splits.zip

--2022-11-09 07:33:48--  https://www.dropbox.com/s/3vdz0mouvex8abd/hpac_splits.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/3vdz0mouvex8abd/hpac_splits.zip [following]
--2022-11-09 07:33:48--  https://www.dropbox.com/s/raw/3vdz0mouvex8abd/hpac_splits.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3bb81458cedb05ca617ac6024b.dl.dropboxusercontent.com/cd/0/inline/BwZFBb5qVOf8CTk7FLESHxQmuXgRub2Ipcv5xBexcYANgaHMAoS6By3Qd-motbmNh2j5pLZqhGm29L8hiHLBFEbOvJ0o2h3VOj45K97AKGuP2jznNRio6MvVW27zWtpZpfKxRmbjLcHWS8GC8_MXep49EDvyshoZS-5o7zMn3aRFVQ/file# [following]
--2022-11-09 07:33:49--  https://uc3bb81458cedb05ca617ac6024b.dl.dropboxusercontent.com/cd/0/inline/BwZFBb5qVOf8CTk7FLESHxQmuXgRub2Ipcv5xBexcYANgaHMAoS6By3Qd-motbmNh2j5

### Данные
Сырые тексты 

In [4]:
!unzip hpac_source

flating: hpac_source/9919705     
  inflating: hpac_source/3590827     
  inflating: hpac_source/2686464     
  inflating: hpac_source/9250145     
  inflating: hpac_source/5006884     
  inflating: hpac_source/7621309     
  inflating: hpac_source/7009404     
  inflating: hpac_source/10564126    
  inflating: hpac_source/7293956     
  inflating: hpac_source/9684908     
  inflating: hpac_source/4812983     
  inflating: hpac_source/10352523    
  inflating: hpac_source/2676018     
  inflating: hpac_source/11159557    
  inflating: hpac_source/7233054     
  inflating: hpac_source/8039783     
  inflating: hpac_source/8139605     
  inflating: hpac_source/8828095     
  inflating: hpac_source/7984796     
  inflating: hpac_source/6503127     
  inflating: hpac_source/5465524     
  inflating: hpac_source/703631      
  inflating: hpac_source/10619753    
  inflating: hpac_source/3674048     
  inflating: hpac_source/9356750     
  inflating: hpac_source/3690034     
  inflating: hpa

In [7]:
!ls hpac_source | wc -l

36225


In [23]:
!unzip hpac_splits

Archive:  hpac_splits.zip
   creating: hpac_corpus/
  inflating: hpac_corpus/hpac_training_128.tsv  
   creating: __MACOSX/
   creating: __MACOSX/hpac_corpus/
  inflating: __MACOSX/hpac_corpus/._hpac_training_128.tsv  
  inflating: hpac_corpus/hpac_dev_128.tsv  
  inflating: __MACOSX/hpac_corpus/._hpac_dev_128.tsv  
  inflating: hpac_corpus/hpac_test_128.tsv  
  inflating: __MACOSX/hpac_corpus/._hpac_test_128.tsv  
  inflating: __MACOSX/._hpac_corpus  


train, test, dev файлы

In [24]:
!du -h --max-depth=1 * | sort -rh

2.8G	hpac_source
951M	hpac_lower_tokenized.zip
46M	hpac_corpus
18M	hpac_splits.zip
2.3M	hw1.ipynb
16K	__MACOSX
12K	__MACOSX/hpac_corpus


## РЕШЕНИЕ: Часть 1. [2 балла] Эксплоративный анализ 
1. Найдите топ-1000 слов по частоте без учета стоп-слов.
2. Найдите топ-10 по частоте: имен, пар имя + фамилия, пар вида ''профессор'' + имя / фамилия. 

In [400]:
import os
import numpy as np
from tqdm.notebook import tqdm
from multiprocessing import Pool, cpu_count
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
sw = stopwords.words('english')

len(os.listdir('hpac_source/'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


36225

### Пункт 1.1

In [190]:
punct = """
!"#$%&()*\+,-\./:;<=>?@\[\]^_`{|}~„“«»†*\—/\-\'‘
"""

In [201]:
def clean_and_get_frequency(doc_name):
    with open(f'hpac_source/{doc_name}') as f:
        text = f.read()
    clean_words = []
    for w in nltk.tokenize.word_tokenize(text):
        w = w.strip(punct)
        if w not in sw and w != '':
            clean_words.append(w)
    return nltk.FreqDist(clean_words)

In [473]:
#!c1.32
with Pool(32) as pool:
    result = list(tqdm(pool.imap(clean_and_get_frequency, os.listdir('hpac_source/')), total=len(os.listdir('hpac_source/'))))

freq = result[0]
for f in tqdm(result[1:]):
    freq += f

KeyboardInterrupt: 

/kernel/lib/python3.8/site-packages/ml_kernel/ignored_keyboard_interrupt.py:16: UserWarning: State committing stage cannot be interrupted. Please wait.
  warnings.warn(self._warn_message)
/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:875: UserWarning: The following variables cannot be serialized: pool
  warnings.warn(message)


In [784]:
freq.most_common(1000)

[("n't", 5819401),
 ('harry', 5302710),
 ('said', 3019712),
 ('would', 2522999),
 ('hermione', 2397976),
 ('could', 2237119),
 ('back', 1847207),
 ('draco', 1828049),
 ('one', 1823068),
 ('like', 1662116),
 ('know', 1561529),
 ('eyes', 1354640),
 ('time', 1336752),
 ('ron', 1201928),
 ('looked', 1180539),
 ('get', 1128294),
 ('asked', 1103905),
 ('well', 1037881),
 ('even', 1019028),
 ('around', 1015126),
 ('see', 985228),
 ('head', 964230),
 ('going', 956226),
 ('think', 949936),
 ('still', 918143),
 ('go', 884351),
 ('severus', 881973),
 ('way', 864164),
 ('face', 859699),
 ('room', 858101),
 ('ginny', 853710),
 ('hand', 834908),
 ('sirius', 833594),
 ('something', 821545),
 ('want', 814858),
 ('thought', 805481),
 ('right', 797753),
 ('potter', 791212),
 ('snape', 784741),
 ('much', 765855),
 ('away', 764919),
 ('two', 746860),
 ('look', 745780),
 ('never', 734960),
 ('really', 698045),
 ('knew', 695257),
 ('first', 685642),
 ('let', 678078),
 ('made', 675815),
 ('good', 667676),
 (

In [478]:
len(freq)

607959

### Пункт 1.2

In [700]:
characters = pd.read_csv('characters_manual_cleaned.csv', sep=';')

In [727]:
characters

,name,description,name_lenght,first_name,last_name
0,Hannah Abbott,Hufflepuff student in Harry Potter's year. Pre...,2,Hannah,Abbott
1,Ludo Bagman,Head of the Department of Magical Games and Sp...,2,Ludo,Bagman
2,Bathilda Bagshot,Author of A History of Magic,2,Bathilda,Bagshot
3,Katie Bell,Gryffindor Quidditch Chaser one year above Har...,2,Katie,Bell
4,Cuthbert Binns,ghost,2,Cuthbert,Binns
...,...,...,...,...,...
203,Scabbers,Ron Weasley's pet rat in the first three stori...,1,Scabbers,
204,Scabior,Snatcher who captures Harry,1,Scabior,
205,Travers,Wizard and Death Eater who escapes from Azkaba...,1,Travers,
206,Trevor,Neville Longbottom's perpetually-misplaced pet...,1,Trevor,


In [783]:
class BigramsCounter:
    
    def __init__(self, dirname='hpac_source/', num_docs=100):
        self.dirname=dirname
        self.num_docs=num_docs
        self.docs = os.listdir(dirname)[:num_docs]
        self.professors_names = self.get_professors_names()
        self.all_names = self.get_all_names()
        self.first_names = list(characters['first_name'].str.lower())
        self.last_names = list(characters.loc[characters['last_name'] != '', 'last_name'].str.lower().unique())
    
    def read_and_clean(self, doc_name) -> list:
        with open(self.dirname + '/' + doc_name) as f:
            text = f.read()
        clean_words = []
        for w in nltk.tokenize.word_tokenize(text):
            w = w.strip(punct)
            if w not in sw and w != '':
                clean_words.append(w)
        return clean_words
        
    def get_name_bigrams(self, doc_name):
        clean_words = self.read_and_clean(doc_name)
        target_bigrams = []
        all_bigrams = list(nltk.bigrams(clean_words))
        for b in all_bigrams:
            if b[0] in self.first_names and b[1] in self.last_names:
                target_bigrams.append(' '.join(b))
        return target_bigrams
    
    def get_names(self, doc_name):
        clean_words = self.read_and_clean(doc_name)
        names = []
        for b in clean_words:
            if b in self.first_names:
                names.append(b)
        return names
    
    def get_professors_bigrams(self, doc_name):
        clean_words = self.read_and_clean(doc_name)
        target_bigrams = []
        all_bigrams = list(nltk.bigrams(clean_words))
        for b in all_bigrams:
            if b[0] == 'professor' and b[1] in target_names:
                target_bigrams.append(' '.join(b))
        return target_bigrams

    def get_professors_names(self):
        """Build up list of professor's names and last names"""
        cond_1 = (characters['description'].str.find('professor') > -1)
        cond_2 = (characters['description'].str.find('teacher') > -1)
        professors = characters.loc[cond_1 | cond_2, 'name'].str.lower()
        return [item for sublist in professors for item in sublist.split()]
        
    def get_all_names(self):
        """Build up list of ALL names and last names from Harry Potter universe"""
        names = characters['name'].str.lower()
        return [item for sublist in names for item in sublist.split()]

#### Популярные имена

In [776]:
#!c1.32
counter = BigramsCounter(num_docs=len(os.listdir('hpac_source/')))
names = []

with Pool(32) as pool:
    for bs in tqdm(pool.imap(counter.get_names, counter.docs), total=counter.num_docs):
        names.extend(bs)

  0%|          | 0/36225 [00:00<?, ?it/s]

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:875: UserWarning: The following variables cannot be serialized: pool
  warnings.warn(message)


In [777]:
#!c1.32
nltk.FreqDist(names).most_common(10)

[('harry', 3991796),
 ('hermione', 1826980),
 ('draco', 1386376),
 ('ron', 904226),
 ('severus', 654520),
 ('ginny', 637945),
 ('sirius', 628686),
 ('james', 467776),
 ('remus', 432662),
 ('lily', 421365)]

#### Популярные имя + фамилия

In [781]:
#!c1.32
counter = BigramsCounter(num_docs=len(os.listdir('hpac_source/')))
bigrams = []

with Pool(32) as pool:
    for bs in tqdm(pool.imap(counter.get_name_bigrams, counter.docs), total=counter.num_docs):
        bigrams.extend(bs)

  0%|          | 0/36225 [00:00<?, ?it/s]

  0%|          | 0/36225 [00:00<?, ?it/s]

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:875: UserWarning: The following variables cannot be serialized: pool
  warnings.warn(message)


In [782]:
nltk.FreqDist(bigrams).most_common(10)

[('harry potter', 134128),
 ('draco malfoy', 50413),
 ('severus snape', 33201),
 ('hermione granger', 31696),
 ('james potter', 29048),
 ('sirius black', 28049),
 ('lucius malfoy', 25130),
 ('albus dumbledore', 22492),
 ('tom riddle', 18947),
 ('remus lupin', 14050)]

#### Популярные профессора


In [636]:
#!c1.32
counter = BigramsCounter(num_docs=len(os.listdir('hpac_source/')))
target_names = counter.professors_names
prof_names = []

with Pool(32) as pool:
    for bs in tqdm(pool.imap(counter.get_professors_bigrams, counter.docs), total=counter.num_docs):
        prof_names.extend(bs)

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:875: UserWarning: The following variables cannot be serialized: pool
  warnings.warn(message)


In [637]:
#!c1.32
nltk.FreqDist(prof_names).most_common(10)

[('professor snape', 46145),
 ('professor mcgonagall', 43183),
 ('professor dumbledore', 24473),
 ('professor flitwick', 9582),
 ('professor slughorn', 5911),
 ('professor sprout', 5796),
 ('professor trelawney', 3193),
 ('professor vector', 1861),
 ('professor quirrell', 1773),
 ('professor severus', 1469)]